In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.20) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed()

class CFG:
    max_len = 256 + 128
    batch_size = 16
    learning_rate = 4e-6
    epochs = 10
    device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

# 데이터 로드 및 전처리 (이전과 동일)
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

train_df.dropna(inplace=True)
train_df.drop_duplicates(subset=['제목', '키워드'], keep='first', inplace=True)

def normalize_title(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text.strip()

def normalize_keywords(text):
    text = re.sub(r'[^가-힣\s,]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.strip()

def clean_keywords(keywords):
    keywords = keywords.split(',')
    keywords = [keyword.strip() for keyword in keywords if keyword.strip()]
    return ' '.join(keywords)

train_df['title'] = train_df['제목'].apply(normalize_title)
train_df['keywords'] = train_df['키워드'].apply(lambda x: clean_keywords(normalize_keywords(x)))
train_df['text'] = train_df['title'] + ' [SEP] ' + train_df['keywords']

test_df['title'] = test_df['제목'].apply(normalize_title)
test_df['keywords'] = test_df['키워드'].apply(lambda x: clean_keywords(normalize_keywords(x)))
test_df['text'] = test_df['title'] + ' [SEP] ' + test_df['keywords']

# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

# 학습 및 검증 데이터 분할
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256+128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item] if self.labels is not None else -1
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [3]:
# KLUE-RoBERTa-large 토크나이저 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import torch

# KLUE-RoBERTa-large 토크나이저 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')
model_path = "klue-roberta-large"  # 실제 경로로 변경하세요

# 설정 로드 및 수정
config = AutoConfig.from_pretrained(model_path)
config.num_labels = len(label_encoder)  # label_encoder는 이전에 정의되어 있어야 합니다

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 모델 생성
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    config=config
)

# 저장된 가중치 로드
state_dict = torch.load(f"{model_path}/pytorch_model.bin")

# 불필요한 키 제거
for key in list(state_dict.keys()):
    if key.startswith('lm_head') or key == 'roberta.embeddings.position_ids':
        del state_dict[key]

# 모델에 가중치 로드
model.load_state_dict(state_dict, strict=False)

# GPU로 모델 이동 (필요한 경우)
model.to(CFG.device)

print("Model loaded successfully!")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_57298/3605669647.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicit

Model loaded successfully!


In [5]:
# 데이터셋 및 데이터로더 생성
train_dataset = TextDataset(train_df.text.tolist(), train_df.label.tolist(), tokenizer, CFG.max_len)
val_dataset = TextDataset(val_df.text.tolist(), val_df.label.tolist(), tokenizer, CFG.max_len)
test_dataset = TextDataset(test_df.text.tolist(), None, tokenizer, CFG.max_len)

train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=CFG.learning_rate)

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [6]:


# 학습 및 검증
for epoch in range(CFG.epochs):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epochs}'):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(CFG.device)
        attention_mask = batch['attention_mask'].to(CFG.device)
        labels = batch['labels'].to(CFG.device)
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch + 1}/{CFG.epochs}, Train Loss: {train_loss / len(train_loader):.4f}")
    
    # 검증
    model.eval()
    val_predictions = []
    val_true_labels = []
    val_loss = 0
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating'):
            input_ids = batch['input_ids'].to(CFG.device)
            attention_mask = batch['attention_mask'].to(CFG.device)
            labels = batch['labels'].to(CFG.device)
            
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()
            
            _, preds = torch.max(outputs.logits, dim=1)
            val_predictions.extend(preds.cpu().tolist())
            val_true_labels.extend(labels.cpu().tolist())
    
    val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
    val_accuracy = accuracy_score(val_true_labels, val_predictions)
    print(f"Validation Loss: {val_loss / len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

# 테스트 데이터 예측
model.eval()
test_predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Predicting'):
        input_ids = batch['input_ids'].to(CFG.device)
        attention_mask = batch['attention_mask'].to(CFG.device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().tolist())

# 예측 결과 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]

# 제출 파일 생성
sample_submission = pd.read_csv("data/sample_submission.csv")
sample_submission["분류"] = decoded_predictions
sample_submission.to_csv("submission_klue_roberta_large.csv", encoding='UTF-8-sig', index=False)

print("Prediction completed and submission file created.")

Epoch 1/10: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2716/2716 [34:41<00:00,  1.30it/s]


Epoch 1/10, Train Loss: 1.2384


Validating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 679/679 [03:08<00:00,  3.61it/s]


Validation Loss: 0.8440, Validation Accuracy: 0.7743, Validation F1 Score: 0.4380


Epoch 2/10: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2716/2716 [34:42<00:00,  1.30it/s]


...

Epoch 10/10, Train Loss: 0.1288


Validating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 679/679 [03:08<00:00,  3.60it/s]


Validation Loss: 0.8224, Validation Accuracy: 0.8224, Validation F1 Score: 0.6608


Predicting: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1463/1463 [06:45<00:00,  3.60it/s]

Prediction completed and submission file created.


In [7]:
# 모델 저장
torch.save(model.state_dict(), 'klue_roberta_large_model.pth')
print("Model saved successfully.")

# 옵티마이저 상태 저장 (선택사항)
torch.save(optimizer.state_dict(), 'optimizer.pth')
print("Optimizer state saved successfully.")

Model saved successfully.
Optimizer state saved successfully.


In [9]:
# 저장된 모델 가중치 불러오기
model.load_state_dict(torch.load('model_save/klue_roberta_large_model.pth'))
model.to(CFG.device)
model.eval()  # 평가 모드로 설정

optimizer.load_state_dict(torch.load('optimizer.pth'))

print("Model and optimizer loaded successfully.")

/tmp/ipykernel_57298/301048072.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_save/klue_roberta_large_model.pth'))
/tmp/ipykerne

Model and optimizer loaded successfully.
